<h1>Decision Tree</h1>

In [1]:
library(mlr)
source('../utils.r')

set.seed(42)

folder_name = '../../raw_data' 
file_name   = 'german.csv'

Loading required package: ParamHelpers
Warning message:
“replacing previous import ‘BBmisc::isFALSE’ by ‘backports::isFALSE’ when loading ‘mlr’”

<h2>1. Dataprep</h2>

In [2]:
data = read.csv(file=sprintf('%s/%s',folder_name,file_name))

In [3]:
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))
head(data)

NRow: 1000
NCol: 22

X,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
1,A11,6,A34,A43,1169,A65,A75,4,A93,⋯,A121,67,A143,A152,2,A173,good,A192,A201,1
2,A12,48,A32,A43,5951,A61,A73,2,A92,⋯,A121,22,A143,A152,1,A173,good,A191,A201,2
3,A14,12,A34,A46,2096,A61,A74,2,A93,⋯,A121,49,A143,A152,1,A172,bad,A191,A201,1
4,A11,42,A32,A42,7882,A61,A74,2,A93,⋯,A122,45,A143,A153,1,A173,bad,A191,A201,1
5,A11,24,A33,A40,4870,A61,A73,3,A93,⋯,A124,53,A143,A153,2,A173,bad,A191,A201,2
6,A14,36,A32,A46,9055,A65,A73,2,A93,⋯,A124,35,A143,A153,1,A172,bad,A192,A201,1


<p style='color: red'> ATENTION: </p>Specifically in R, in classification problem, target must be set as Factor.

In [4]:
data$V21 = as.factor(data$V21)

MLR works only with features and target, this means that others columns must be dorped.

In [5]:
drops = c('X')
data  = data[ , !(names(data) %in% drops)]
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))

NRow: 1000
NCol: 21

<h2>2. Modeling</h2>

Function
```R
makeLearner(cl, id = cl, predict.type = "response", predict.threshold = NULL, 
            fix.factors.prediction = FALSE, ..., par.vals = list(), config = list())
```
Param.:

* cl: [character(1)] Class of learner. By convention, all classification learners start with “classif.”. A list of all integrated learners is available on the learners help page < https://mlr-org.github.io/mlr-tutorial/release/html/integrated_learners/ >.
* predict: [character(1)] “response” (= labels) or “prob” (= probabilities and labels by selecting the ones with maximal probability). Default is “response”.
* par.vals: [list] Optional list of named (hyper)parameters. The arguments in ... take precedence over values in this list. We strongly encourage you to use one or the other to pass (hyper)parameters to the learner but not both.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner <br>

In [6]:
cl = "classif.ksvm"

List all parameters that can be used in this classifier.
The value must be set in par.vals parameter.

In [7]:
getParamSet(cl)

                       Type  len    Def
scaled              logical    -   TRUE
type               discrete    -  C-svc
kernel             discrete    - rbfdot
C                   numeric    -      1
nu                  numeric    -    0.2
epsilon             numeric    -    0.1
sigma               numeric    -      -
degree              integer    -      3
scale               numeric    -      1
offset              numeric    -      1
order               integer    -      1
tol                 numeric    -  0.001
shrinking           logical    -   TRUE
class.weights numericvector <NA>      -
fit                 logical    -   TRUE
cache               integer    -     40
                                                Constr Req Tunable Trafo
scaled                                               -   -    TRUE     -
type              C-svc,nu-svc,C-bsvc,spoc-svc,kbb-svc   -    TRUE     -
kernel        vanilladot,polydot,rbfdot,tanhdot,lap...   -    TRUE     -
C                           

In [8]:
learner = makeLearner(cl = cl
                     , predict.type = "prob"
                     , par.vals = list()
                     )

Function
```R
makeClassifTask(id = deparse(substitute(data)), data, target, weights = NULL, blocking = NULL, 
                positive = NA_character_, fixup.data = "warn", check.data = TRUE)
```
Param.:

* data: [data.frame] A data frame containing the features and target variable(s).
* target: [character(1)] Name of the target variable.
* positive: [character(1)] Positive class for binary classification (otherwise ignored and set to NA). Default is the first factor level of the target attribute.
* fixup.data: [character(1)] Should some basic cleaning up of data be performed? Currently this means removing empty factor levels for the columns. Possible coices are: “no” = Don't do it. “warn” = Do it but warn about it. “quiet” = Do it but keep silent. Default is “warn”.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [9]:
task = makeClassifTask( data = data
                      , target = 'V21'
                      , positive = '2'
                      , fixup.data = 'no'
                      )

Function:
```R
makeResampleDesc(method, predict = "test", ..., stratify = FALSE, stratify.cols = NULL)
```
Param.:

* method: [character(1)] “CV” for cross-validation, “LOO” for leave-one-out, “RepCV” for repeated cross-validation, “Bootstrap” for out-of-bag bootstrap, “Subsample” for subsampling, “Holdout” for holdout.
* predict: What to predict during resampling: “train”, “test” or “both” sets. Default is “test”.
* ... : [any] Further parameters for strategies.
    * iters [integer(1)] Number of iterations, for “CV”, “Subsample” and “Boostrap”.
    * split [numeric(1)] Proportion of training cases for “Holdout” and “Subsample” between 0 and 1. Default is 2/3.
    * reps [integer(1)] Repeats for “RepCV”. Here iters = folds * reps. Default is 10.
    * folds [integer(1)] Folds in the repeated CV for RepCV. Here iters = folds * reps. Default is 10.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeResampleDesc

In [10]:
resample = makeResampleDesc( method = "CV"
                           , iters = 10
                           , predict = 'both'
                           , stratify = FALSE
                           )

List of performance measures:

Doc.: http://mlr-org.github.io/mlr-tutorial/release/html/measures/

In [11]:
measures = list(mmce #MMCE 
               ,acc  #acuracia
               ,f1   #f1
               ,ppv  #precision
               ,tpr  #recall
               ,auc  #AUC
               ,gini #Gini
               #,timetrain #tempo execucao
               )

Function:
```R
resample(learner, task, resampling, measures, weights = NULL, models = FALSE, extract, 
         keep.pred = TRUE, ..., show.info = getMlrOption("show.info"))
```
Param.:

* learner: [Learner] The learner.
* task: [Task] The task.
* resampling: [ResampleInstance] Resampling strategy.
* measures: [Measure | list of Measure] Performance measure(s) to evaluate. Default is mean misclassification error (mmce)
* weights: [numeric] Optional, non-negative case weight vector to be used during fitting. If given, must be of same length as observations in task and in corresponding order. Overwrites weights specified in the task. By default NULL which means no weights are used unless specified in the task.
* models: [logical(1)] Should all fitted models be returned? Default is FALSE.
* keep.pred: [logical(1)] Keep the prediction data in the pred slot of the result object. If you do many experiments (on larger data sets) these objects might unnecessarily increase object size / mem usage, if you do not really need them. In this case you can set this argument to FALSE. Default is TRUE.
* show.info: [logical(1)] Print verbose output on console? Default is set via configureMlr.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/resample

In [12]:
r = resample(learner = learner
            ,task = task 
            ,resampling = resample 
            ,measures = measures
            #---------------------#
            ,models = TRUE
            ,keep.pred = FALSE
            ,show.info = TRUE
            )

[Resample] cross-validation iter 1: mmce.test.mean=0.23,acc.test.mean=0.77,f1.test.mean=0.511,ppv.test.mean= 0.6,tpr.test.mean=0.444,auc.test.mean=0.746,gini.test.mean=0.492
[Resample] cross-validation iter 2: mmce.test.mean=0.27,acc.test.mean=0.73,f1.test.mean=0.509,ppv.test.mean=0.636,tpr.test.mean=0.424,auc.test.mean=0.776,gini.test.mean=0.552
[Resample] cross-validation iter 3: mmce.test.mean=0.26,acc.test.mean=0.74,f1.test.mean= 0.5,ppv.test.mean=0.684,tpr.test.mean=0.394,auc.test.mean=0.806,gini.test.mean=0.612
[Resample] cross-validation iter 4: mmce.test.mean=0.22,acc.test.mean=0.78,f1.test.mean=0.542,ppv.test.mean=0.812,tpr.test.mean=0.406,auc.test.mean=0.792,gini.test.mean=0.585
[Resample] cross-validation iter 5: mmce.test.mean=0.21,acc.test.mean=0.79,f1.test.mean=0.588,ppv.test.mean=0.714,tpr.test.mean= 0.5,auc.test.mean=0.789,gini.test.mean=0.577
[Resample] cross-validation iter 6: mmce.test.mean=0.29,acc.test.mean=0.71,f1.test.mean=0.383,ppv.test.mean=0.562,tpr.test.mean=

<h2>3. Result Analysis</h2>

Train Measures

In [13]:
r$measures.train

iter,mmce,acc,f1,ppv,tpr,auc,gini
1,0.1511111,0.8488889,0.7130802,0.8407960,0.6190476,0.9146584,0.8293169
2,0.1500000,0.8500000,0.7019868,0.8548387,0.5955056,0.9171060,0.8342120
3,0.1500000,0.8500000,0.7019868,0.8548387,0.5955056,0.9154552,0.8309104
4,0.1633333,0.8366667,0.6711409,0.8379888,0.5597015,0.9160330,0.8320659
5,0.1622222,0.8377778,0.6726457,0.8522727,0.5555556,0.9142975,0.8285949
6,0.1555556,0.8444444,0.6846847,0.8685714,0.5650558,0.9206605,0.8413211
7,0.1655556,0.8344444,0.6542923,0.8597561,0.5280899,0.9083728,0.8167457
8,0.1633333,0.8366667,0.6711409,0.8620690,0.5494505,0.9102769,0.8205537
9,0.1577778,0.8422222,0.6885965,0.8674033,0.5709091,0.9177716,0.8355433
10,0.1633333,0.8366667,0.6783370,0.8333333,0.5719557,0.9144838,0.8289677


Test Measures

In [14]:
r$measures.test

iter,mmce,acc,f1,ppv,tpr,auc,gini
1,0.23,0.77,0.5106383,0.6000000,0.4444444,0.7458143,0.4916286
2,0.27,0.73,0.5090909,0.6363636,0.4242424,0.7761194,0.5522388
3,0.26,0.74,0.5000000,0.6842105,0.3939394,0.8059701,0.6119403
4,0.22,0.78,0.5416667,0.8125000,0.4062500,0.7922794,0.5845588
5,0.21,0.79,0.5882353,0.7142857,0.5000000,0.7885714,0.5771429
6,0.29,0.71,0.3829787,0.5625000,0.2903226,0.7587658,0.5175316
7,0.22,0.78,0.6333333,0.7037037,0.5757576,0.8692899,0.7385798
8,0.20,0.80,0.5238095,0.7333333,0.4074074,0.8127854,0.6255708
9,0.20,0.80,0.5238095,0.6470588,0.4400000,0.7936000,0.5872000
10,0.22,0.78,0.5925926,0.6400000,0.5517241,0.7625061,0.5250121


Train Aggregated Result

In [15]:
apply(r$measures.train,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.1582222 0.8417778 0.6837892 0.8531868 0.5710777 0.9149116 0.8298232

Test Aggregated Result

In [16]:
apply(r$measures.test,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.2320000 0.7680000 0.5306155 0.6733956 0.4434088 0.7905702 0.5811404

Run Time in seconds

In [17]:
r$runtime

[1] 4.146915

<h2>4. Prediction for new data</h2>

Read the data to predict

In [18]:
new.data = read.csv(file=sprintf('%s/%s', folder_name, file_name))

Search for the best model in crossvalidation and use it to score the incoming data

In [19]:
best.model = which.max(r$measures.test$acc)

In [20]:
pred = predict(r$models[[best.model]], newdata = new.data)

Prediction Result

In [21]:
pred

Prediction: 1000 observations
predict.type: prob
threshold: 1=0.50,2=0.50
time: 0.13
  truth    prob.1     prob.2 response
1     1 0.9066882 0.09331181        1
2     2 0.4330402 0.56695979        2
3     1 0.8862412 0.11375880        1
4     1 0.4809901 0.51900991        2
5     2 0.3049037 0.69509631        2
6     1 0.7780743 0.22192566        1
... (1000 rows, 4 cols)


Cast result to data.frame to access the prediction

In [22]:
head(as.data.frame(pred))

truth,prob.1,prob.2,response
1,0.9066882,0.09331181,1
2,0.4330402,0.56695979,2
1,0.8862412,0.11375880,1
1,0.4809901,0.51900991,2
2,0.3049037,0.69509631,2
1,0.7780743,0.22192566,1
